# Práctica 3.5
### [Introducción a los sistemas inteligentes](https://fagonzalezo.github.io/)
___________

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/pHasW3C30nxzXzjLQl1k) antes del final de la clase. El archivo debe nombrarse como  `isi-practica3.5-unalusername.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad.

___________

## Cuadrado de Rubik (o Cubo de Rubik 2D)

El *Cuadrado de Rubik* es un rompecabezas inspirado en el famoso cubo de Rubik. Consiste de un arreglo de 16 fichas organizadas en una matriz de $4\times 4$ como se ilustra en la siguiente figura:

<img src="https://raw.githubusercontent.com/fagonzalezo/iis-2018-2/master/rubik2D.png"
alt="Cuadrado de Rubik " width="240" height="180" border="10" />

Los colores son ilustrativos, lo importante es el número en cada una de las fichas. Se pueden hacer 10 movimientos diferentes correspondientes a rotar las 4 fichas alrededor de cada uno de los puntos A, B, C, D y E en el sentido de las manecillas del reloj o en el sentido opuesto.

Su objetivo es modelar el *Cuadrado de Rubik* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Cuadrado de Rubik

Un Cuadrado de Rubik debe representarse como una lista con valores enteros que representan cada una de las fichas.

Por ejemplo un Cuadrado de Rubik resuelto debe verse así:

```py
[ 1,  2,  3,  4,
  5,  6,  7,  8,
  9, 10, 11, 12,
 13, 14, 15, 16]
```

#### Definición de acciones

La siguiente lista define las posibles acciones que se pueden ejecutar:

In [1]:
'''
These values MUST not be changed.
They represent the movements of the Rubik's Square.
'''
ACTIONS = ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

Cada acción indica la posición y el sentido. Por ejemplo, `'C-'` rota la posición C en el sentido opuesto de las manecillas del reloj. Si aplicamos esta acción al estado solución se obtiene el estado:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 10, 12,
 13, 14, 15, 16]
```

Si sobre este estado, aplicamos la acción `'E+'` obtenemos:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 15, 10,
 13, 14, 16, 12]
```

#### Clase Rubik2D_problem

In [2]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class Rubik2d_problem(Problem):

    def __init__(self, initial):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15]
        [1, 9, 4, 8, 5, 6, 3, 2, 15, 10, 11, 12, 13, 14, 7, 16]
        [2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12]
        '''
        self.expanded = 0
        self.goal = tuple([i for i in range(1, 17)])  # Goal state is numbers 1-16 in order
        super().__init__(initial=tuple(initial), goal=self.goal)

    def actions(self, state):
        """Return a list of actions that can be executed in the given
        state."""
        return ACTIONS  # Using the predefined actions list ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

    def result(self, state, action):
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        new_state = list(state)  # Create a copy to avoid modifying the original state

        # Define which positions are affected by each rotation point
        rotations = {
            'A': [0, 1, 5, 4],    # Top-left rotation point
            'B': [2, 3, 7, 6],    # Top-right rotation point
            'C': [5, 6, 10, 9],   # Middle rotation point
            'D': [8, 9, 13, 12],  # Bottom-left rotation point
            'E': [10, 11, 15, 14] # Bottom-right rotation point
        }

        point = action[0]  # Get the rotation point (A, B, C, D, or E)
        direction = action[1]  # Get the direction (+ or -)
        affected_positions = rotations[point]

        if direction == '+':  # Clockwise rotation
            values = [state[i] for i in affected_positions]
            values = [values[-1]] + values[:-1]  # Rotate right
        else:  # Counter-clockwise rotation
            values = [state[i] for i in affected_positions]
            values = values[1:] + [values[0]]

        for pos, val in zip(affected_positions, values):
            new_state[pos] = val

        return tuple(new_state)  # Return the new state as a tuple

    def is_goal(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        return state == self.goal

    def action_cost(self, s, a, s1):
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        return 1  # Uniform cost for all actions

### 2. Evalue su código con diferentes estrategias de búsqueda



Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice las implementaciones de búsqueda en profundidad y búsqueda en profundidad iterativa.

Evaluelo para ver cuál es la máxima profundidad que se puede alcanzar en un tiempo razonable con cada estrategia de búsqueda. Reporte los resultados.

In [3]:
#Class Node for the algorithms from AIMA
import math

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

#Data structures for the algorithms
from collections import deque
import heapq

FIFOQueue = deque
LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [4]:
import sys

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    tot_expand = 0
    for limit in range(1, sys.maxsize):
        result , tot = depth_limited_search(problem, limit)
        tot_expand += tot
        if result != cutoff:
            return result, tot_expand
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    tot_expand = 0
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node, tot_expand
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            tot_expand+=1
            for child in expand(problem, node):
                frontier.append(child)
    return result, tot_expand

def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def bfs(problem):
    """
    Perform a breadth-first search on the problem.
    Return the list of actions to reach the goal state.
    """
    ### your code here ###
    r = breadth_first_search(problem)
    return path_actions(r) if r != failure else []

def iterativeDeepeningSearch(problem):
    """
    Perform an iterative deepening search on the problem.
    Return the list of actions to reach the goal state.
    """
    ### your code here ###
    r,_ = iterative_deepening_search(problem)
    return path_actions(r) if r != failure else []

def iterativeDeepeningCountExpand(problem):
    """
    Perform an iterative deepening search on the problem.
    Return the list of actions to reach the goal state.
    """
    ### your code here ###
    _,t = iterative_deepening_search(problem)
    return t


# Creates a problem instance to simulate some moves
problem = Rubik2d_problem(list(range(1, 17)))
state = problem.initial
print(state)
state = problem.result(state, "A+")
print(state)
state = problem.result(state, "B-")

# Now you can test the search algorithms
problem = Rubik2d_problem(state)
actions = bfs(problem)
print(actions)
actions = iterativeDeepeningSearch(problem)
print(actions)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
(5, 1, 3, 4, 6, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
['A-', 'B+']
['B+', 'A-']


Try the following problem. What happens?

In [5]:
problem = Rubik2d_problem( [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15])

state = problem.initial

actions = iterativeDeepeningSearch(problem)
print(actions)

['E+', 'C-', 'D-', 'B-']


### 3. Implemente diferente heurísticas para el problema

Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice la implementación de A*.

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de la 3ra edición de [Russell10].

In [6]:
def nullHeuristic(state):
    return 0

def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    tot_expanded = 0
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node , tot_expanded
        tot_expanded+=1
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure, tot_expanded

def g(n): return n.path_cost

def aStarSearch(problem, h=nullHeuristic):
    """
    Perform an A*t search on the problem.
    Returns the list of actions to reach the goal state.
    """
    h = h or problem.h
    r , _ = best_first_search(problem, f=lambda n: g(n) + h(n))
    return path_actions(r) if r != failure else []

def aStarCountExpanded(problem, h=nullHeuristic):
    h = h or problem.h
    _ , t = best_first_search(problem, f=lambda n: g(n) + h(n))
    return t

def myHeuristicMissplacedElements(state):
    missplaced = 0
    for i in range(len(state.state)):
        if state.state[i]!=i+1:
            missplaced+=1
    return missplaced

def myHeuristicManhattan(state):
    """
    Computes the sum of Manhattan distances of each tile from its current position
    to its goal position in a 4x4 grid.
    """
    size = 4  # Dimensions of the board (4x4)
    distance = 0
    for index, value in enumerate(state.state):
        if value == 0:
            continue  # Skip the empty tile if it exists
        # Current row and column of the tile
        current_row, current_col = divmod(index, size)
        # Goal row and column of the tile
        goal_row, goal_col = divmod(value - 1, size)
        # Add the Manhattan distance for this tile
        distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance

In [7]:
problem = Rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print(f'IDS: {iterativeDeepeningSearch(problem)}')
print(f'A* null heuristic: {aStarSearch(problem, nullHeuristic)}')
print(f'A* missplaced elements heuristic: {aStarSearch(problem, myHeuristicMissplacedElements)}')
print(f'A* manhattan distance heuristic: {aStarSearch(problem, myHeuristicManhattan)}')

IDS: ['E-', 'D-', 'D-', 'B+', 'A+', 'C+']
A* null heuristic: ['D+', 'D+', 'B+', 'E-', 'A+', 'C+']
A* missplaced elements heuristic: ['A+', 'B+', 'E-', 'D+', 'D+', 'C+']
A* manhattan distance heuristic: ['A+', 'B+', 'D+', 'D+', 'E-', 'C+']


Ahora contamos los nodos expandidos en cada algoritmo

In [8]:
problem = Rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print(f'IDS: {iterativeDeepeningCountExpand(problem)}')
print(f'A* null heuristic: {aStarCountExpanded(problem, nullHeuristic)}')
print(f'A* missplaced elements heuristic: {aStarCountExpanded(problem, myHeuristicMissplacedElements)}')
print(f'A* manhattan distance heuristic: {aStarCountExpanded(problem, myHeuristicManhattan)}')

IDS: 10215
A* null heuristic: 59858
A* missplaced elements heuristic: 13
A* manhattan distance heuristic: 6


Es evidente que hay heurisiticas que mejoran el resultado del desempeño de $A*$, haremos un ejercicio de probar todas las posibles configuraciones que me dan una solucion y promediamos su expansion

In [ ]:
import random
from scipy.optimize import root_scalar

def generate_test_cases(min_depth = 1, max_depth = 7 , test_per_case =  10, seed = 42):
    random.seed(seed)
    s = []
    for i in range(min_depth, max_depth+1):
        for _ in range(test_per_case):
            problem = Rubik2d_problem(list(range(1, 17)))
            state = problem.initial
            for j in range(i):
                state = problem.result(state, random.choice(ACTIONS))
            s.append(state)
    return s

def effective_branching_factor(N, d):
    if d == 0 or N <= 1:
        return 0
    
    def f(b):
        try:
            return (b**(d + 1) - 1) / (b - 1) - N
        except ZeroDivisionError:
            return float('inf')

    
    lower_bound = -1
    upper_bound = max(2, N)
    
    try:
        sol = root_scalar(f, method='brentq', bracket=[lower_bound, upper_bound])
        if sol.converged:
            return sol.root
        else:
            return float('nan')
    except Exception as e:
        print(f"Error in effective_branching_factor: {e}")
        print(f(lower_bound),f(upper_bound))
        return float('nan')
                

def getCounter(init):
    problem = Rubik2d_problem(list(init))
    IDSPath = iterativeDeepeningSearch(problem)
    if len(IDSPath) == 0:
        return None
    d = len(IDSPath)

    # Obtener nodos expandidos
    ids = iterativeDeepeningCountExpand(problem)
    aStarNull = aStarCountExpanded(problem, nullHeuristic)
    aStarMissplaced = aStarCountExpanded(problem, myHeuristicMissplacedElements)
    aStarManhattan = aStarCountExpanded(problem, myHeuristicManhattan)

    return {
        ('d',''): d,
        ('Expanded Nodes', 'IDS'): ids,
        ('Expanded Nodes', 'aStarNull'): aStarNull,
        ('Expanded Nodes', 'aStarMissplaced'): aStarMissplaced,
        ('Expanded Nodes', 'aStarManhattan'): aStarManhattan,
        ('Effective Branch Factor', 'IDS'): effective_branching_factor(ids, d),
        ('Effective Branch Factor', 'aStarNull'): effective_branching_factor(aStarNull, d),
        ('Effective Branch Factor', 'aStarMissplaced'): effective_branching_factor(aStarMissplaced, d),
        ('Effective Branch Factor', 'aStarManhattan'): effective_branching_factor(aStarManhattan, d)
    }

In [10]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

columns = pd.MultiIndex.from_tuples([
    ('d', ''),
    ('Expanded Nodes', 'IDS'),
    ('Expanded Nodes', 'aStarNull'),
    ('Expanded Nodes', 'aStarMissplaced'),
    ('Expanded Nodes', 'aStarManhattan'),
    ('Effective Branch Factor', 'IDS'),
    ('Effective Branch Factor', 'aStarNull'),
    ('Effective Branch Factor', 'aStarMissplaced'),
    ('Effective Branch Factor', 'aStarManhattan')
])

cases = generate_test_cases()
d = []
for i,case in enumerate(cases):
    n = getCounter(case)
    if n is not None: d.append(n)
    if (i+1)%5==0: print(f'{i+1}/{len(cases)} processed')

df = pd.DataFrame(d, columns=columns)
df

5/70 processed
10/70 processed
15/70 processed
20/70 processed
25/70 processed
30/70 processed
35/70 processed
40/70 processed
45/70 processed
50/70 processed
55/70 processed
60/70 processed
65/70 processed
70/70 processed


d Expanded Nodes                                           \
                 IDS aStarNull aStarMissplaced aStarManhattan   
0   1              1         1               1              1   
1   1              1         6               1              1   
2   1              1         5               1              1   
3   1              1         2               1              1   
4   1              1         2               1              1   
.. ..            ...       ...             ...            ...   
62  3             15        93               3              3   
63  3             51        85               3              3   
64  5           4611      6255              55              5   
65  3             18       266               6              3   
66  5           5723      6937              33            105   

   Effective Branch Factor                                           
                       IDS aStarNull aStarMissplaced aStarManhattan  
0                 0.000000  0.000000        0.000000       0.000000  
1                 0.000000  5.000000        0.000000       0.000000  
2                 0.000000  4.000000        0.000000       0.000000  
3                 0.000000  1.000000        0.000000       0.000000  
4                 0.000000  1.000000        0.000000       0.000000  
..                     ...       ...             ...            ...  
62                2.000000  4.136083        0.810536       0.810536  
63                3.296844  4.000000        0.810536       0.810536  
64                5.177534  5.519326        1.934386       0.926519  
65                2.164997  6.057334        1.278163       0.810536  
66                5.417563  5.640077        1.699279       2.260221  

[67 rows x 9 columns]

A continuación mostramos las tablas con las estadisticas de cantidad de nodos expandidos y factor de ramificacion dado un tamaño fijo de solucion $d$

In [11]:
result = df.groupby('d').mean().round(2)
print('Mean values of the amount of nodes expanded')
result

Mean values of the amount of nodes expanded


Expanded Nodes                                           \
             IDS aStarNull aStarMissplaced aStarManhattan   
d                                                           
1           1.00      4.31            1.00           1.00   
2           6.42     40.25            2.08           2.00   
3          42.72    221.39            4.94           3.00   
4         486.45   1212.45           40.73           6.64   
5        3856.11   6422.11           26.22          27.78   
6       23493.25  28165.50          147.75          13.00   

  Effective Branch Factor                                           
                      IDS aStarNull aStarMissplaced aStarManhattan  
d                                                                   
1                    0.00      3.31            0.00           0.00  
2                    1.81      5.62            0.65           0.62  
3                    2.92      5.51            1.07           0.81  
4                    4.33      5.52            1.82           1.07  
5                    4.89      5.43            1.43           1.29  
6                    5.02      5.24            1.57           1.13

In [12]:
result = df.groupby('d').min().round(2)
print('Min values of the amount of nodes expanded')
result

Min values of the amount of nodes expanded


Expanded Nodes                                           \
             IDS aStarNull aStarMissplaced aStarManhattan   
d                                                           
1              1         1               1              1   
2              3        13               2              2   
3             15        75               3              3   
4            234       454               4              4   
5           1565      2358               5              5   
6           9399     13859               6              6   

  Effective Branch Factor                                           
                      IDS aStarNull aStarMissplaced aStarManhattan  
d                                                                   
1                    0.00      0.00            0.00           0.00  
2                    1.00      3.00            0.62           0.62  
3                    2.00      3.82            0.81           0.81  
4                    3.61      4.32            0.89           0.89  
5                    4.12      4.49            0.93           0.93  
6                    4.40      4.71            0.95           0.95

In [13]:
result = df.groupby('d').max().round(2)
print('Max values of the amount of nodes expanded')
result

Max values of the amount of nodes expanded


Expanded Nodes                                           \
             IDS aStarNull aStarMissplaced aStarManhattan   
d                                                           
1              1         9               1              1   
2             11        58               3              2   
3            110       379              14              3   
4            895      2158             128             17   
5           7194     10859              61            105   
6          38867     49450             511             28   

  Effective Branch Factor                                           
                      IDS aStarNull aStarMissplaced aStarManhattan  
d                                                                   
1                    0.00      8.00            0.00           0.00  
2                    2.70      7.07            1.00           0.62  
3                    4.40      6.87            1.94           0.81  
4                    5.18      6.54            3.05           1.64  
5                    5.68      6.19            1.98           2.26  
6                    5.63      5.87            2.61           1.45

Con estos resultados, es evidente la mejoria en el desempeño del algoritmo de $A*$ al usar las heurísticas propuestas (Manhattan y cantidad de elementos mal ubicados), lo cual es consistente con los resultados presentados en la figura del libro